## Reinforcement Learning: A bit of theory

------------------------------------------------------------------------------------
#### Notations and terminology

**Supervised learning** relies on labels of target outcomes for training models to predict these outcomes on unseen data.
**Unsupervised learning** does not need labels; it learns inherent patterns on training data to group unseen data according to these learned patterns.
Motivations behind *Reinforcement Learning* are indeed cases in which you cannot define a complete supervision and can only define feedback signals based on actions taken, and cases in which you’d like to learn the optimal decisions to make as time progresses.

To define an **RL** framework, you need to define a goal for a learning system (the **agent**) that makes decisions in an **environment**. Then you need to translate this goal into a mathematical formula called a **reward** function, aimed at rewarding or penalizing the agent when it takes an action and acting as a feedback loop to help the agent reach the predefined goal. There are three key RL components: **state**, **action**, and **reward**; at each time step, the agent receives a representation of the environment’s state $\large s_t $, takes an action $\large a_t $, and receives a numerical reward $\large r_{t+1}$.

The general RL problem consists of learning entire sequences of actions, called *policies*, and noted $\large \pi(a|s) $, to maximize reward.

We can formalize the objective of learning when the goal is to maximize the cumulative reward after a time step $\large t $ by defining the concept of cumulative return $\large G_t $:
$$ \large G_t = r_{t+1} + r_{t+2} + ... + r_{T} $$

When there is a notion of a final step, each subsequence of actions is called an **episode**, and the [MDP](https://en.wikipedia.org/wiki/Markov_decision_process) is said to have a **finite horizon**. If the agent-environment interaction goes on forever, the MDP is said to have an **infinite horizon**; in which case you can introduce a concept of discounting to define a mathematically unified notation for cumulative return:
$$ \large G_t = \sum_k^T \gamma^k r_{t+1+k} $$

If $\large \gamma = 1 $ and $\large T $ is finite, this notation is the finite horizon formula for $\large G_t $, while if $\large \gamma < 1 $, episodes naturally emerge even if $\large T $ is infinite because terms with a large value of $\large k $  become exponentially insignificant compared to terms with a smaller value of $\large k $. Thus the sum converges for infinite horizon MDP, and this notation applies for both finite and infinite horizon MDP.

------------------------------------------------------------------------------------
#### Policy-based RL

In a policy-based RL, the policy is defined as a parametric function of some parameters $\large \theta $: $ \large \pi_\theta = \pi(a|s,\theta) $ and estimated directly by standard gradient methods. This is done by defining a performance measure, typically the expected value of $\large G_t $ for a given policy, and applying gradient ascent to find $\large \theta $ that maximizes this performance measure. $ \large \pi_\theta $ can be any parameterized functional form of $\large \pi $ as long as it is differentiable with respect to $\large \theta $; thus a neural network can be used to estimate it, in which case the parameters $\large \theta $ are the weights of the neural network and the neural network predicts entire policies based on input states.

Direct policy search is relatively simple and also very limited because it attempts to estimate an entire policy directly from a set of previously experienced policies. Combining policy-based RL with value-based RL to refine the estimate of $\large G_t $ almost always improves accuracy and convergence of RL.

------------------------------------------------------------------------------------
#### Value-based RL

An MDP consists in a sequence $ (s_0, a_0, r_1, s_1, a_1, r_2, …, s_n) $, or more generally $ (s_t, a_t, r_{t+1}, s_{t+1})_n $, and the dynamics of an MDP is fully characterized by a transition probability $ T(s,a,s’) = P(s’|s,a) $ that defines the probability for any state $\large s $ to going to any other state $\large s’ $, and a reward function $ r(s,a) = P(r|s,a) $ that defines the expected reward for any given state $\large s $. The goal of RL is to learn the best policies to maximize $\large G_t $.

To evaluate a policy, define a measure for any state s called a state **value function** $\large V(s) $ that estimates how good it is to be in s for a particular way of acting, that is, for a particular policy π:

$$\large v_{\pi}(s) = \mathbb{E_\pi}(G_t|s)$$
$$\large v_{\pi}(s) = \mathbb{E_\pi}(\sum_k^T \gamma^k r_{t+1+k}|s)$$
$$\large v_{\pi}(s) = \mathbb{E_\pi}(r_{t+1} + G_{t+1}|s)$$

Replacing the expectation formula by a sum over all probabilities yields the **Bellman equation**:

$$\large v_{\pi}(s) = \sum_a \pi(a | s) \sum_{s'} T(s,a,s’)[r(s,a) + \gamma v_{\pi}(s')]$$

You can estimate $ T $ and $ r $ by counting all the occurrences of observed transitions and rewards in a set of observed interactions between the agent and the environment. $ T $ and $ r $ define the model of the MDP.

We can extend the notion of state value function to the notion of **state-action value function** $\large Q(s,a) $ to define an optimal policy:

$$\large Q_{\pi}(s,a) = \sum_{s'} T(s,a,s’)[r(s,a) + \gamma Q_{\pi}(s',a')]$$

$$\large Q^{\ast}_{\pi}(s,a) = \sum_{s'} T(s,a,s’)[r(s,a) + \gamma \max_{a'} Q^{\ast}_{\pi}(s',a')]$$

The latter is called the **Bellman Optimality equation**; it estimates the value of taking a particular action $\large a $ in a particular state $\large s $ assuming you’ll take the best actions thereafter. If you transform the Bellman Optimality equation into an assignment function you get an iterative algorithm that, assuming you know $\large T $ and $\large r $, is guaranteed to converge to the optimal policy with random sampling of all possible actions in all states. Because you can apply this equation to states in any order, this iterative algorithm is referred to as **asynchronous dynamic programming**.

For systems in which you can easily compute or sample $\large T $ and $\large r $, this approach is sufficient and is referred to as **model-based** RL because you know the transition dynamics. But for most real-case problems, it is more convenient to produce stochastic estimates of the Q-values based on experience accumulated so far, so the RL agent learns in real time at every step and ultimately converges toward true estimates of the Q-values. To this end, combine asynchronous dynamic programming with the concept of moving average:
$$\large Q^{\ast}_{k+1}(s,a) = Q^{\ast}_{k}(s,a) + \alpha(G_t(s) - Q^{\ast}_{k}(s,a)) $$

and replace $\large G_t(s) $ by a sampled value as defined in the Bellman Optimality equation:
$$\large Q^{\ast}_{k+1}(s,a) = Q^{\ast}_{k}(s,a) + \alpha(r_{t+1} + \gamma \max_{a'} Q^{\ast}_k(s',a') - Q^{\ast}_{k}(s,a)) $$

The latter is called **temporal difference learning**; it enables you to update the moving average $\large Q^{\ast}_{k+1}(s,a) $ based on the difference between $ Q^{\ast}_{k}(s,a) $ and $ Q^{\ast}_{k}(s',a') $. This post shows a one-step temporal difference, as used in the most straightforward version of the **Q-learning algorithm**, but you could extend this equation to include more than one step (n-step temporal difference). Again, some theorems exist that prove Q-learning converges to the optimal policy $\large \pi^{\ast} $ assuming infinite random action selection.

------------------------------------------------------------------------------------
#### RL action-selection strategy

You can alleviate the infinite random action selection condition by using a more efficient random action selection strategy such as **ε-Greedy** to increase sampling of states frequently encountered in good policies and decrease sampling of less valuable states. In ε-Greedy, the agent selects a random action with probability $\large ε $, and the rest of the time (that is, with probability $\large 1-ε $), the agent selects the best action according to the latest Q-values, which is defined as $ argmax_{a} Q(s,a) $. Generally, $\large ε $ is chosen to be large at the beginning to favor exploration of state-action space, and progressively reduced to a smaller value.

------------------------------------------------------------------------------------
#### Combining deep learning and reinforcement learning

In many cases, for example when playing chess or Go, the number of states, actions, and combinations thereof is so large that the memory and time needed to store the array of Q-values is enormous. There are more combinations of states and actions in the game Go than known stars in the universe. Instead of storing Q-values for all states and actions in an array which is impossible for Go and many other cases, deep RL attempts to generalize experience from a subset of states and actions to new states and actions. 

Deep Q-learning uses a supervised learning approximation to $ Q(s,a) $ by using $ r_{t+1} + \gamma \max_{a'} Q^{\ast}_k(s',a') $ as the label, because the Q-learning assignment function is equivalent to a gradient update of the general form $ x = x – ∝∇J $(for any arbitrary x) where:
$$\large J = \dfrac{1}{2} (Q^{\ast}_{k}(s,a) - (r_{t+1} + \gamma \max_{a'} Q^{\ast}_k(s',a')))^2 $$ 
The latter is called the **Square Bellman Loss**; it allows you to estimate Q-values based on generalization from mapping states and actions to Q-values using a deep learning network, hence the name deep Q-learning.

Deep Q-learning is not guaranteed to converge anymore because the label used for supervised learning depends on current values of the network weights, which are themselves updated based on learning from the labels, hence a problem of broken ergodicity. But this approach is often good enough in practice and can generalize to the most complex RL problems (such as autonomous driving, robotics, or playing Go). 

In deep RL, the training set changes at every step, so you need to define a buffer to enable batch training, and schedule regular refreshes as experience accumulates for the agent to learn in real time. This is called **experience replay** and can be compared to the process of learning while dreaming in biological systems: experience replay allows you to keep optimizing policies over a subset of experiences $ (s_t, a_t, r_{t+1}, s_{t+1})_n $ kept in memory.

------------------------------------------------------------------------------------
#### Definitions

$$\large q_{\pi}(s_t, a_t) = \sum_{t'=t}^T \mathbb{E}_{\pi_\theta}[r(s_{t'}, a_{t'}) | s_t, a_t]$$

$$\large v_{\pi}(s_t) = \sum_{t'=t}^T \mathbb{E}_{\pi_\theta}[r(s_{t'}, a_{t'}) | s_t]$$

$$\large p_{\theta}(s_1,a_1,...,s_T,a_T) = p(s_1) \prod_{t=1}^T \pi_{\theta}(a_t|s_t)p(s_{t+1}|s_t,a_t)$$

------------------------------------------------------------------------------------
#### Bellman Expectation Equations

$$\large v_{\pi}(s) = \mathbb{E}[q_{\pi}(s, a)]$$

$$\large v_{\pi}(s) = \sum_a \pi(a | s) q_{\pi}(s, a)$$

$$\large q_{\pi}(s, a) = \sum_{s', r} p(s', r | s, a)[r + \gamma v_{\pi}(s')]$$

$$\large v_{\pi}(s) = \sum_a \pi(a | s) \sum_{s', r} p(s', r | s, a)[r + \gamma v_{\pi}(s')]$$

$$\large q_{\pi}(s, a) = \sum_{s', r} p(s', r | s, a)[r + \gamma \sum_{a'} \pi(a' | s') q_{\pi}(s', a')]$$

------------------------------------------------------------------------------------
#### Bellman Optimality Equations

$$\large v_{\ast}(s) = \max_a q_{\ast}(s, a)$$

$$\large q_{\ast}(s, a) = \sum_{s', r} p(s', r | s, a)[r + \gamma v_{\ast}(s')]$$

$$\large v_{\ast}(s) = \max_a \sum_{s', r} p(s', r | s, a)[r + \gamma v_{\ast}(s')]$$

$$\large q_{\ast}(s, a) = \sum_{s', r} p(s', r | s, a)[r + \gamma \max_{a'} q_{\ast}(s', a')]$$

------------------------------------------------------------------------------------
#### Policy Improvement Theorem

$$\large q_\pi(s, \pi'(s)) \geq v_\pi(s) \implies v_{\pi'}(s) \geq v_{\pi}(s) $$

In [1]:
system(`jupyter-nbconvert --output-dir=./ --to HTML rl-theory.ipynb`)

['[NbConvertApp] Converting notebook rl-theory.ipynb to HTML',
 '[NbConvertApp] Writing 663061 bytes to rl-theory.html']